# Boreholes

This notebook shows how to plot borehole data along with a groundwater measurement well.

Two different packages are used:
- `brodata`, to obtain and visualise borehole data from the BRO database
- `geost` to get Bodemkundig booronderzoek from the BRO database

In [ ]:
import contextily as ctx
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point

import hydropandas as hpd

In [ ]:
# reading data from a groundwatermonitoring well (see notebook 01)
gw_bro = hpd.GroundwaterObs.from_bro("GMW000000041261", 1)

#### Borehole (Brodata)

Brodata can be installed using:

`pip install brodata`

In [ ]:
import brodata

In [ ]:
# select extent with a buffer around gw_bro
p = Point(gw_bro.x, gw_bro.y)
extent = np.array(p.buffer(1000).bounds).astype(int)[[0, 2, 1, 3]]

# get bro ids of borehole data
gdf = brodata.bhr.get_characteristics(extent=extent)

In [ ]:
# plot borehole data
f, axes = plt.subplots(ncols=2, figsize=(16, 8))

gdf["broid"] = gdf.index.str[-7:]
gdf["x"] = gdf.geometry.x
gdf = gdf.sort_values("x")
gdf["geometry"] = gdf.geometry
line = LineString(gdf["deliveredLocation"].values)
distance_along_line = pd.Series(
    [line.project(point) for point in gdf["geometry"]], gdf.index
)

# specify colors
colors = brodata.plot.lithology_colors.copy()
colors["sterkZandigSilt"] = (243, 225, 100)
colors["siltigZandMetGrind"] = (220, 200, 80)
colors["siltigZand"] = (243, 225, 150)
colors["zwakGrindigZand"] = (231, 210, 70)

# plot values
xticks = []
xticklabels = []
for bro_id in gdf.index:
    bhrgt = brodata.bhr.GeotechnicalBoreholeResearch.from_bro_id(bro_id)
    for x, bl in enumerate(bhrgt.descriptiveBoreholeLog):
        brodata.plot.bro_lithology(
            bl["layer"],
            x=distance_along_line[bro_id],
            z=bhrgt.offset,
            drilling=bro_id,
            ax=axes[0],
            linewidth=20,
            colors=colors,
        )
        xticks.append(distance_along_line[bro_id])
        xticklabels.append(bro_id[-7:])

axes[0].set_xticks(xticks)
axes[0].set_xticklabels(xticklabels, rotation=45, ha="right")
axes[0].set_ylabel("z (m t.o.v. NAP)")
# plot map

# plot well
axes[1].scatter(gw_bro.x, gw_bro.y, color="black")
axes[1].annotate(gw_bro.name, (gw_bro.x + 25, gw_bro.y + 10))

# plot boreholes
legend_labels = gdf["broid"].tolist()
gdf.plot("broid", ax=axes[1], legend=True)

# plot line
gpd.GeoDataFrame(geometry=[line]).plot(ax=axes[1])

# settings
axes[1].set_xlim(extent[:2])
axes[1].set_ylim(extent[2:])

# basemap
ctx.add_basemap(axes[1], crs=28992)

#### Bodemkundig booronderzoek (geost)

The geost package can only be pip installed for python version 3.12 or higher. Install it 
using:

`pip install geost`

In [ ]:
!pip install geost

In [ ]:
import geost

In [ ]:
# get Bodemkundig booronderzoek (BHR-GT)
boreholes = geost.bro_api_read("BHR-GT", bbox=[int(i) for i in p.buffer(1000).bounds])

In [ ]:
# add colors based on column 'standard_name'
col = "geotechnicalSoilName"  # "standardSoilName"
boreholes.data.df["color"] = boreholes.data.df[col].map(colors)

# plot borehole data
f, axes = plt.subplots(figsize=(16, 8), ncols=2)
labels = []
for i, (xy, df) in enumerate(boreholes.data.df.groupby("x")):
    df["top"] = df["surface"] - df["top"]
    df["bottom"] = df["surface"] - df["bottom"]
    for _, row in df.iterrows():
        color = [c / 255 for c in row["color"]] + [1.0]
        if row[col] in labels:
            axes[0].fill_between(
                [i - 0.25, i + 0.25],
                [row["top"], row["top"]],
                [row["bottom"], row["bottom"]],
                color=color,
            )
        else:
            axes[0].fill_between(
                [i - 0.25, i + 0.25],
                [row["top"], row["top"]],
                [row["bottom"], row["bottom"]],
                color=color,
                label=row[col],
            )
            labels.append(row[col])
axes[0].set_xticks(
    np.arange(0, i + 1), boreholes.data.df.sort_values("x")["nr"].unique()
)
# Rotate and align the xtick labels
for label in axes[0].get_xticklabels():
    label.set_rotation(45)
    label.set_ha("right")

if boreholes.header.vertical_reference == 5709:
    axes[0].set_ylabel("m NAP")
axes[0].set_xlim(-0.5, i + 1.5)
axes[0].legend()
axes[0].set_title(f"Bodemkundig booronderzoek close to {gw_bro.name}")
# plot map

# plot well
axes[1].scatter(gw_bro.x, gw_bro.y, label=gw_bro.name, color="black")

# plot boreholes
for i, dfnr in boreholes.data.df.groupby(["x", "y"]):
    axes[1].scatter(*i, label=dfnr["nr"].unique()[0])

# settings
axes[1].set_xlim(extent[:2])
axes[1].set_ylim(extent[2:])
axes[1].legend()

# basemap
ctx.add_basemap(axes[1], crs=28992)